# Microcalcification cluster creation
This notebook contains the functions used to create microcalcification, microcalcification clusters, placement of clusters in breast phantom, and preprocessing of Monte-Carlo GPU simulations.

To convert any calcification or cluster into a raw file, use: 

`cluster_type.tofile('cluster{0}x{0}_{1}_nm{2}.raw'.format(cluster_size, index, num_calcs))`

In [ ]:
import numpy as np
import random as rand
import math
from raster_geometry import * #For creating spherical calcifications
#!pip install bresenham
from bresenham import bresenham #For creating linear clusters

## Creating individual calcifications
### Spherical calcifications

In [ ]:
def create_calc(size, num_rmv_calc):
    '''
    Creates one microcalcification. 
    
    Arguments: 
        size - diameter of calcification.
        num_rmv_calc - how many voxels to remove from the outer surface.
        
    Returns:
        calc_init (ndarray) - calcification of size x size x size. 
    '''
    #Method from raster_geometry. 
    calc_init=sphere(size, size//2).astype(int)
    
    #Indexes of voxels on the outer surface of the calcification. 
    #Includes voxels on the top of the array and the side of the array.
    #num_rmv_calc specifies how many voxels are removed. 
    tops = [(ii, i, j)  for i in range(size) for ii in range(size) 
            if i==0 or i==size-1 for j in range(size)]
    sides = [(ii, i, jj) for jj in [0, size-1] for i in range(size) for ii in range(size)]
    tot=set(tops+sides)
    
    to_rmv=rand.sample(tot, int(num_rmv_calc))
    for ind in to_rmv: 
        calc_init[ind[0], ind[1], ind[2]]=0
    return calc_init

### Rod-like calcifications

In [ ]:
def create_calc_rod(size, num_rmv_calc):
    #creating with all vertical, and no smoothing
    #calc_init= np.ones((size, size, size))
    axis=-1
    #if rand.uniform(0,1) < 0.5:
    #    axis=-2
    calc_init=cylinder(size, size, size//3, axis=axis, smoothing=None).astype(int)
    #top of array - sides are side of array - total - set of tuples that index 
    #indexes of voxels on the surface of the calcification
    tops = [(ii, i, j)  for i in range(size) for ii in range(size) 
            if i==0 or i==size-1 for j in range(size)]
    sides = [(ii, i, jj) for jj in [0, size-1] for i in range(size) for ii in range(size)]
    tot=set(tops+sides)
    to_rmv=rand.sample(tot, int(num_rmv_calc))
    for ind in to_rmv: 
        calc_init[ind[0], ind[1], ind[2]]=0
    return calc_init

## Creating microcalcification clusters
### Random clusters

In [ ]:
def create_cluster(cluster_size, num_calcs, min_calc_size=3, max_calc_size=9):
    '''
    Creates random calcification cluster.
    Arguments:
        cluster_size - voxel length of one side of cluster. (Specification for MC-GPU: 1 vx^3 = 70 micromm.)
        num_calcs - how many individual microcalcifications located inside the cluster.
        min_calc_size - smallest microcalcification size, default = 3 voxels. 
        max_calc_size - largest microcalcification size, default = 9 voxels.
        
    Returns:
        cluster_type (ndarray) - cluster with <num_calcs> randomly placed calcifications. 
        
    '''
    
    calc_size_list = [rand.randint(min_calc_size, max_calc_size) for i in range(num_calcs)]
    
    cluster_init= np.zeros((cluster_size, cluster_size, cluster_size))
    cluster_ind=[(i, j, k) for i in range(cluster_size-max_calc_size) 
            for j in range(cluster_size-max_calc_size) for k in range(cluster_size-max_calc_size)]
    #calc_place_ind - need to find places to place the calcifcation cluster that don't overlap 
    calc_place_ind=rand.sample(cluster_ind, num_calcs)
    
    for calc_size, ind in zip(calc_size_list, calc_place_ind):
        calc_num_rmv = rand.randint(int(calc_size**3 * 0.2), int(calc_size**3 * 0.3))
        calc=create_calc(calc_size, calc_num_rmv) #remove number of voxels = calc size squared (3x3x3 - remove 3^2=9)
        cluster_init[ind[0]:ind[0]+calc_size, ind[1]:ind[1]+calc_size, ind[2]:ind[2]+calc_size]=calc
    
    cluster_type = cluster_init.astype('uint8')
    #cluster_type.tofile('cluster{0}x{0}_{1}_nm{2}.raw'.format(cluster_size, index, num_calcs))
    return cluster_type

### Random non-uniform clusters

In [ ]:
def create_cluster_nonuniform(cluster_size, num_calcs, min_calc_size=3, max_calc_size=9, weights=
                                                                      [0.2, 0.2, 0.2, 0.2, 0.03, 0.03, 0.03]):
    '''
    Creates random calcification cluster with non-uniform distribution of sizes.
    Arguments:
        cluster_size - voxel length of one side of cluster. (Specification for MC-GPU: 1 vx^3 = 70 micromm.)
        num_calcs - how many individual microcalcifications located inside the cluster.
        min_calc_size - smallest microcalcification size, default = 3 voxels. 
        max_calc_size - largest microcalcification size, default = 9 voxels.
        weights (list) - distribution of sizes of the microcalcifications. From left to right, should correspond
            with the min_calc_size up to the max_calc_size. Default = [0.2, 0.2, 0.2, 0.2, 0.03, 0.03, 0.03], where
            the larger sizes, 7-9 voxels, have a probability of 10%. 
        
    Returns:
        cluster_type (ndarray) - cluster with <num_calcs> randomly placed calcifications with non-uniform distribution 
            of sizes. 
    '''
    
    calc_size_list = rand.choices([i for i in range(min_calc_size, max_calc_size+1)], weights, k=num_calcs)
    
    cluster_init= np.zeros((cluster_size, cluster_size, cluster_size))
    cluster_ind=[(i, j, k) for i in range(cluster_size-max_calc_size) 
            for j in range(cluster_size-max_calc_size) for k in range(cluster_size-max_calc_size)]
    #calc_place_ind - need to find places to place the calcifcation cluster that don't overlap 
    calc_place_ind=rand.sample(cluster_ind, num_calcs)
    
    for calc_size, ind in zip(calc_size_list, calc_place_ind):
        calc_num_rmv = rand.randint(int(calc_size**3 * 0.2), int(calc_size**3 * 0.3))
        calc=create_calc(calc_size, calc_num_rmv) #remove number of voxels = calc size squared (3x3x3 - remove 3^2=9)
        cluster_init[ind[0]:ind[0]+calc_size, ind[1]:ind[1]+calc_size, ind[2]:ind[2]+calc_size]=calc
    
    cluster_type = cluster_init.astype('uint8')
    #cluster_type.tofile('cluster{0}x{0}_{1}_nm{2}.raw'.format(cluster_size, index, num_calcs))
    return cluster_type

### Linear clusters

In [ ]:
def create_cluster_linear(cluster_size, num_calcs, min_calc_size = 3, max_calc_size = 7, num_away_min = 10, num_away_max = 30):
    
    distance=0
    length = int(math.sqrt(2) * cluster_size) - 50 #170
    print(length)
    x = 0
    y = 0
    xx = 0
    yy = 0
    i=0
    
    while distance not in range(length-10, length+10):
        x=rand.randint(max_calc_size, cluster_size-max_calc_size)
        y=rand.randint(max_calc_size, cluster_size-max_calc_size)
        xx=rand.randint(max_calc_size, cluster_size-max_calc_size)
        yy=rand.randint(max_calc_size, cluster_size-max_calc_size)
        distance=math.sqrt((x-xx)**2+(y-yy)**2)
    print(x, y)
    print(xx, yy)


    coordinates = list(bresenham(x, y, xx, yy))

    calc_size_list = [rand.randint(min_calc_size, max_calc_size) for i in range(num_calcs)]

    calc_place_ind=rand.sample(coordinates, num_calcs)


    cluster_init= np.zeros((cluster_size, cluster_size, cluster_size))
    for calc_size, indd in zip(calc_size_list, calc_place_ind):

        ind = (indd[0]+rand.randint(num_away_min, num_away_max)*rand.randrange(-1,2, 1), indd[1]+rand.randint(num_away_min, num_away_max)*rand.randrange(-1,2, 1))
        while not (ind[0] in range(0, cluster_size) and 
                   ind[0]+calc_size in range(0, cluster_size)) or not (ind[1] in range(0, cluster_size) and ind[1]+calc_size in range(0, cluster_size)):
            ind = (indd[0]+rand.randint(num_away_min, num_away_max)*rand.randrange(-1,2, 1), indd[1]+rand.randint(num_away_min, num_away_max)*rand.randrange(-1,2, 1))

        calc_num_rmv = rand.randint(int(calc_size**3 * 0.2), int(calc_size**3 * 0.3))

        calc=create_calc(calc_size, calc_num_rmv) #remove number of voxels = calc size squared (3x3x3 - remove 3^2=9)


        if ind[0]+calc_size and ind[1]+calc_size < cluster_size:
            cluster_init[50:50+calc_size, ind[0]:ind[0]+calc_size, ind[1]:ind[1]+calc_size]=calc
            i = i + 1

    print("done")        
    cluster_type = cluster_init.astype('uint8')
    return cluster_type

### Clusters with rod-calcs with vertical and no smoothing

In [ ]:
def create_cluster_rod(cluster_size, num_calcs, min_calc_size=3, max_calc_size=9):
    #min_calc_size = 3
    #max_calc_size = 9
    
    calc_size_list = [rand.randint(min_calc_size, max_calc_size) for i in range(num_calcs)]
    
    cluster_init= np.zeros((cluster_size, cluster_size, cluster_size))
    cluster_ind=[(i, j, k) for i in range(cluster_size-max_calc_size) 
            for j in range(cluster_size-max_calc_size) for k in range(cluster_size-max_calc_size)]
    #calc_place_ind - need to find places to place the calcifcation cluster that don't overlap 
    calc_place_ind=rand.sample(cluster_ind, num_calcs)
    
    for calc_size, ind in zip(calc_size_list, calc_place_ind):
        calc_num_rmv = calc_size**2#rand.randint(int(calc_size**3 * 0.2), int(calc_size**3 * 0.3))
        calc=create_calc_rod(calc_size, calc_num_rmv) #remove number of voxels = calc size squared (3x3x3 - remove 3^2=9)
        cluster_init[ind[0]:ind[0]+calc_size, ind[1]:ind[1]+calc_size, ind[2]:ind[2]+calc_size]=calc
    
    cluster_type = cluster_init.astype('uint8')
    #cluster_type.tofile('cluster{0}x{0}_{1}_nm{2}.raw'.format(cluster_size, index, num_calcs))
    return cluster_type

### Clusters with rod-like and spherical calcs 

In [ ]:
def create_cluster_rod_sphere(cluster_size, num_calcs, min_calc_size=3, max_calc_size=9):
    #min_calc_size = 3
    #max_calc_size = 9
    
    calc_size_list = [rand.randint(min_calc_size, max_calc_size) for i in range(num_calcs)]
    
    cluster_init= np.zeros((cluster_size, cluster_size, cluster_size))
    cluster_ind=[(i, j, k) for i in range(cluster_size-max_calc_size) 
            for j in range(cluster_size-max_calc_size) for k in range(cluster_size-max_calc_size)]
    #calc_place_ind - need to find places to place the calcifcation cluster that don't overlap 
    calc_place_ind=rand.sample(cluster_ind, num_calcs)
    
    for calc_size, ind in zip(calc_size_list, calc_place_ind):
        if rand.uniform(0,1) < 0.5:
            calc_num_rmv = calc_size**2#rand.randint(int(calc_size**3 * 0.2), int(calc_size**3 * 0.3))
            calc=create_calc_rod(calc_size, calc_num_rmv) #remove number of voxels = calc size squared (3x3x3 - remove 3^2=9)
        else:
            calc_num_rmv = rand.randint(int(calc_size**3 * 0.2), int(calc_size**3 * 0.3))
            calc=create_calc(calc_size, calc_num_rmv) #remove number of voxels = calc size squared (3x3x3 - remove 3^2=9)
        cluster_init[ind[0]:ind[0]+calc_size, ind[1]:ind[1]+calc_size, ind[2]:ind[2]+calc_size]=calc
    
    cluster_type = cluster_init.astype('uint8')
    #cluster_type.tofile('cluster{0}x{0}_{1}_nm{2}.raw'.format(cluster_size, index, num_calcs))
    return cluster_type

# Placing calcifications inside breast phantom
## Placing 5 and 10 mm sizes cluster (71 vx and 142 vx) 

In [ ]:
x_cent=2159//2
y_cent=1680//2
i_cent=601//2


phantom_name="cluster_malignant_only_{0}_{1}nm_m_non.raw"
path = '/cadt/simulation/mcgpu/30mm_thk_phntm/pce_1764975963_crop.raw'
#b3 is a new phantom
path_clusters = '/cadt/simulation/calc_cluster_creation/'

parameters_list = [(71, 40), (142, 40)]

for i in range(len(parameters_list)): 
   
    parameters = parameters_list[i]
    cluster_size = parameters[0]
    num_calcs = parameters[1]
    start_i = i_cent-cluster_size//2    
    
    bb = np.fromfile(path, dtype='uint8')
    bb=bb.reshape(601, 2159, 1680)
    start_y=y_cent - 700 
    #cluster = np.fromfile(path_clusters+name, dtype='uint8')
    #cluster=cluster.reshape(cluster_size,cluster_size,cluster_size)

    for ii in range(3): #number of rows
        change_x=x_cent-600#650 #one thing we are changing for the positioning of the clusters 
        for iii in range(4):#4): #number of clusters on each row 
            #cluster = np.fromfile(path_clusters+name, dtype='uint8')
            #cluster=cluster.reshape(100,100,100)
            cluster = create_cluster_nonuniform(cluster_size, num_calcs, 3, 9)
            for i in range(cluster_size):
                for x in range(cluster_size):
                    for y in range(cluster_size):
                        if cluster[i,x,y] == 1:
                            bb[start_i+i, change_x+x, start_y+y]=250
            change_x=change_x+350 #300 #space between each cluster is 400 voxels 
        start_y = start_y+400
    bb.tofile(phantom_name.format(cluster_size, num_calcs))
    print(phantom_name.format(cluster_size, num_calcs))

## Placing 20 mm sized clusters (285 vx)

In [ ]:
x_cent=2159//2
y_cent=1680//2
i_cent=601//2


phantom_name="cluster_malignant_only_{0}_{1}nm_s.raw"
path = '/cadt/simulation/mcgpu/30mm_thk_phntm/pce_1764975963_crop.raw'
#b3 is a new phantom
path_clusters = '/cadt/simulation/calc_cluster_creation/'

parameters_list = [(285, 10), (285,20), (285,30), (285,40), (285, 50)]

for i in range(len(parameters_list)):    
    
    parameters = parameters_list[i]
    cluster_size = parameters[0]
    num_calcs = parameters[1]
    start_i = i_cent-cluster_size//2    
    
    bb = np.fromfile(path, dtype='uint8')
    bb=bb.reshape(601, 2159, 1680)
    start_y=y_cent - 700 
    #cluster = np.fromfile(path_clusters+name, dtype='uint8')
    #cluster=cluster.reshape(cluster_size,cluster_size,cluster_size)

    for ii in range(3): #number of rows
        change_x=x_cent-600#650 #one thing we are changing for the positioning of the clusters 
        for iii in range(3):#4): #number of clusters on each row 
            #cluster = np.fromfile(path_clusters+name, dtype='uint8')
            #cluster=cluster.reshape(100,100,100)
            cluster = create_cluster(cluster_size, num_calcs, 3, 5)
            for i in range(cluster_size):
                for x in range(cluster_size):
                    for y in range(cluster_size):
                        if cluster[i,x,y] == 1:
                            bb[start_i+i, change_x+x, start_y+y]=250
            change_x=change_x+450 #300 #space between each cluster is 400 voxels 
        start_y = start_y+400
    bb.tofile(phantom_name.format(cluster_size, num_calcs))
    print(phantom_name.format(cluster_size, num_calcs))

# Pre-processing after MC-GPU

In [ ]:
def preprocess(path, five=400000, other=400000, lower=25, upper=100):
    #path includes img_name - is a posix path or just a path 
    path = pathlib.Path(path)
    raw = np.fromfile(path, dtype='float32', sep="")
    #im assuming float32 is 32-bit real 

    image_size = [2]  # Assumes that there are always 2 images in the RAW image! 
    #2 images in MC-GPU mammogram (1st - primary&scattered xrays, 2nd - primary only)

    image_size.extend((1500, 3000))
    raw = raw.reshape(image_size)

    im_temp = raw[0, :, :] #!!might need to change 
    #im_temp = np.rot90(im_temp)  # Rotate the image

    # Finds locations where we want to look at - when we did not invert it yet
    if re.compile('^.*5.*$').search(path.name) != None:
    #if img_name_l == 'prj_30mm_2_cluster_malignant_5mm_{}.raw.gz.raw':
        im_mask= np.greater(im_temp, five)
        print('five')
    else:
        im_mask= np.greater(im_temp, other)   # finds locations we want to look at
        print('other')
    #true and false if the areas are greater than 500000
    #before another test value 2.6e06, 500000


    #prob have to lower than 500000 
    im_temp = np.multiply(im_temp, im_mask) #-> if false, then it equals 0 
    im_temp = np.max(im_temp) - im_temp  # invert the image
    im_temp = im_temp + 1
    im_temp = np.log(im_temp)  # Take the log of the image

    #----- zet minimum where it is greater than 14 - picks smallest value that is greater than 14 
    # Get the minimum where it is greater than 14
    im_min = np.min(im_temp[im_temp > 14])

    # Perform a shift such that the im_min is 0
    im_temp = im_temp - im_min
    im_temp[im_temp < 0] = 0

    # Zero out the outside of the images again
    im_temp = np.multiply(im_temp, im_mask)
    im_temp[im_temp == np.max(im_temp)] = 0 #zeros out pixels outside of the breast

    p1, p2 = np.percentile(im_temp[im_temp > 0.7], (lower, upper))

    test = exposure.rescale_intensity(im_temp, in_range=(p1, p2)) * 255  # Edit this value to change the image bits
    return test

In [ ]:
def write_out(test, img_name, add='_full_0.7_25'):
    #'prj_30mm_2_cluster_malignant_calc_size_5mm{}.raw.gz.raw'
    test.astype('uint8').tofile(img_name.format(add)) #saving to a RAW file
    #choosing to use the values greater than 0.7 and keeping the 25th to 100th percentile 
    
    img = Image.frombytes('L', (3000, 1500), test.astype('uint8'))
    img.save(img_name.format(add)+'.png') #saving to a PNG file 

For processing several images at once.

In [ ]:
path = pathlib.Path('/cadt/simulation/mcgpu/finished/')
for currentFile in path.iterdir():
    if currentFile.suffix == '.raw':
        print(currentFile)
        test = preprocess(currentFile)
        img_name = currentFile.name[:-11] + '{}' + currentFile.name[-11:]
        write_out(test, img_name)
        print(img_name)